In [1]:
import regex
import numpy as np
import pandas as pd
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
import spacy
import pyLDAvis

/Users/evandimartinis/opt/anaconda3/lib/python3.8/site-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)
/Users/evandimartinis/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_least_angle.py:34: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  method='lar', copy_X=True, eps=np.finfo(np.float).eps,
/Users/evandimartinis/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_mo

In [2]:
import nltk
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

/Users/evandimartinis/opt/anaconda3/lib/python3.8/site-packages/sklearn/feature_extraction/image.py:172: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dtype=np.int):
/Users/evandimartinis/opt/anaconda3/lib/python3.8/site-packages/scipy/io/matlab/mio5.py:95: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and 

In [3]:
df = pd.read_csv('CleanedWorldNews.csv')
df.head(10)

,Unnamed: 0,Unnamed: 0.1,title,score,id,url,coms_num,author,upvotes,Unnamed: 0.1.1,word_count
0,0,0,gaza apartment block collapses airstrike,139,na81sp,https://www.yahoo.com/news/gaza-apartment-bloc...,89,Home0nWater,139,NaN,6
1,1,1,us sanctions 7 hezbollah finances,31,naoiw0,https://www.al-monitor.com/originals/2021/05/u...,4,FeydSeswatha982,31,NaN,6
2,2,2,unicef calls uk give 20 vaccines countries,6,nan3v6,https://www.theguardian.com/society/2021/may/1...,8,edifsego,6,NaN,12
3,3,3,covid19 bodies float ganga river ballia ghazip...,15,natz0q,https://thewire.in/rights/now-bodies-float-dow...,5,nram76,15,NaN,16
4,4,4,foreign secretary issues warning russia ransom...,9,naw0qt,https://www.bbc.com/news/technology-57084943,1,HydrolicKrane,9,NaN,8
5,5,5,greece offers 300000 reward killers strangled ...,45,nae6h5,https://www.theguardian.com/world/2021/may/11/...,5,cardart,45,NaN,14
6,6,6,hamas fired 15 rockets toward dimona home isra...,85,nanx1k,https://www.timesofisrael.com/liveblog_entry/s...,72,TransGerman,85,NaN,11
7,7,7,1 dead 3 severely injured largest ever hamas b...,4580,n9vmk4,https://www.ynetnews.com/article/ULALGRFGI,3074,Mdk_251,4580,NaN,13
8,8,8,conor mcgregor tops forbes top 10 list worlds ...,2,navz0e,https://www.forbes.com/sites/forbespr/2021/05/...,5,SpareThisOne2thPls,2,NaN,16
9,9,9,school attack means afghanistans future,12,najdw3,https://www.npr.org/2021/05/11/995948176/what-...,2,Maxcactus,12,NaN,8


In [4]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations
        
data = df['title'].tolist()
data_words = list(sent_to_words(data))

print(data_words[0])

['gaza', 'apartment', 'block', 'collapses', 'airstrike']


In [5]:
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
bigram_mod = gensim.models.phrases.Phraser(bigram)
print(bigram_mod[data_words[0]])

['gaza', 'apartment', 'block', 'collapses', 'airstrike']


In [6]:
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]


def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

data_words_nostops = remove_stopwords(data_words)
data_words_bigrams = make_bigrams(data_words_nostops)
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[0])

['gaza', 'apartment', 'block', 'collapse', 'airstrike']


In [14]:
id2word = corpora.Dictionary(data_lemmatized)
texts = data_lemmatized
corpus = [id2word.doc2bow(text) for text in texts]
print(corpus[0])
print(id2word)

[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1)]
Dictionary(1079 unique tokens: ['airstrike', 'apartment', 'block', 'collapse', 'gaza']...)


In [12]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=2, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)
# Print the KeywordS in the 20 topics
print(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0, '0.019*"covid" + 0.012*"india" + 0.009*"find" + 0.008*"china" + 0.008*"say" + 0.007*"warn" + 0.006*"war" + 0.006*"new" + 0.006*"country" + 0.006*"vaccine"'), (1, '0.021*"israel" + 0.020*"gaza" + 0.019*"kill" + 0.015*"say" + 0.013*"israeli" + 0.011*"report" + 0.010*"rocket" + 0.009*"covid" + 0.009*"hama" + 0.009*"airstrike"')]


In [13]:
import pyLDAvis.gensim_models
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda_model, corpus, id2word, mds='mmds')
vis

/Users/evandimartinis/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/pairwise.py:57: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dtype = np.float
/Users/evandimartinis/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/pairwise.py:57: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dtype = np.float
/Users/evandimartinis/

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1      0.093592 -0.079176       1        1  50.308221
0     -0.093592  0.079176       2        1  49.691779, topic_info=         Term       Freq      Total Category  logprob  loglift
4        gaza  32.000000  32.000000  Default  30.0000  30.0000
107      kill  30.000000  30.000000  Default  29.0000  29.0000
41     israel  37.000000  37.000000  Default  28.0000  28.0000
91     report  17.000000  17.000000  Default  27.0000  27.0000
44     rocket  16.000000  16.000000  Default  26.0000  26.0000
..        ...        ...        ...      ...      ...      ...
197  violence   8.083508  13.636230   Topic2  -5.2870   0.1764
121       say  12.631388  36.736882   Topic2  -4.8407  -0.3683
371     study   6.816959  10.348269   Topic2  -5.4575   0.2819
62      world   7.120520  16.671739   Topic2  -5.4139  -0.1514
73      woman   6.368022  11.533598   Topic2  -5.5256   0.1054

[120 rows x 6 columns], token_table=      Topic      Freq         Term
term                              
63        1  0.910444  afghanistan
392       1  0.944633          air
0         1  0.967054    airstrike
133       2  0.939492       amazon
452       1  0.950631     antitank
...     ...       ...          ...
62        1  0.599817        world
62        2  0.419872        world
293       1  0.976070     xinjiang
213       1  0.214182         year
213       2  0.856730         year

[109 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 1])